# Домашнее задание № 11. Машинный перевод

## Задание 1 (6 баллов + 2 доп балла).
Нужно обучить трансформер на этом же или на другом корпусе (можно взять другую языковую пару с того же сайте) и оценивать его на всей тестовой выборке (а не на 10 примерах как сделал я).

Чтобы получить 2 доп балла вам нужно будет придумать как оптимизировать функцию translate. Подсказка: модель может предсказывать батчами.


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! unzip /content/drive/MyDrive/translations.zip

Archive:  /content/drive/MyDrive/translations.zip
  inflating: Translations/opus.en-ru-test.ru.txt  
  inflating: Translations/opus.en-ru-test.en.txt  
  inflating: Translations/opus.en-ru-dev.en.txt  
  inflating: Translations/tokenizer_ru  
  inflating: Translations/opus.en-ru-dev.ru.txt  
  inflating: Translations/tokenizer_en  
  inflating: Translations/opus.en-ru-train.en.txt  
  inflating: Translations/opus.en-ru-train.ru.txt  


In [2]:
! pip install tokenizers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 54.4 MB/s eta 0:00:00


**Готовим токенизаторы для трансформера**

In [3]:
en_sents_train = open('/content/drive/MyDrive/Translations/opus.en-ru-train.en.txt').read().splitlines()
ru_sents_train = open('/content/drive/MyDrive/Translations/opus.en-ru-train.ru.txt').read().splitlines()

en_sents_dev = open('/content/drive/MyDrive/Translations/opus.en-ru-dev.en.txt').read().splitlines()
ru_sents_dev = open('/content/drive/MyDrive/Translations/opus.en-ru-dev.ru.txt').read().splitlines()

en_sents_test = open('/content/drive/MyDrive/Translations/opus.en-ru-test.en.txt').read().splitlines()
ru_sents_test = open('/content/drive/MyDrive/Translations/opus.en-ru-test.ru.txt').read().splitlines()

In [4]:
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.trainers import BpeTrainer

tokenizer_en = Tokenizer(BPE())
tokenizer_en.pre_tokenizer = Whitespace()
trainer_en = BpeTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"])
#tokenizer_en.train(files=["/content/Translations/opus.en-ru-train.en.txt"], trainer=trainer_en)

tokenizer_ru = Tokenizer(BPE())
tokenizer_ru.pre_tokenizer = Whitespace()
trainer_ru = BpeTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"])
#tokenizer_ru.train(files=["/content/Translations/opus.en-ru-train.ru.txt"], trainer=trainer_ru)

In [13]:
# tokenizer_en.save('/content/drive/MyDrive/Translations/tokenizer_en')
# tokenizer_ru.save('/content/drive/MyDrive/Translations/tokenizer_ru')
tokenizer_en = Tokenizer.from_file("/content/drive/MyDrive/Translations/tokenizer_en")
tokenizer_ru = Tokenizer.from_file("/content/drive/MyDrive/Translations/tokenizer_ru")

**Код для подгрузки данных через генератор**

In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data

class Dataset(torch.utils.data.Dataset):

    def __init__(self, texts_en, texts_ru, PAD_IDX):
        self.texts_en = [torch.LongTensor(sent) for sent in texts_en]
        self.texts_en = torch.nn.utils.rnn.pad_sequence(self.texts_en, padding_value=PAD_IDX)

        self.texts_ru = [torch.LongTensor(sent) for sent in texts_ru]
        self.texts_ru = torch.nn.utils.rnn.pad_sequence(self.texts_ru, padding_value=PAD_IDX)

        self.length = len(texts_en)

    def __len__(self):
        return self.length

    def __getitem__(self, index):

        ids_en = self.texts_en[:, index]
        ids_ru = self.texts_ru[:, index]

        return ids_en, ids_ru

**Архитектура модели**

In [15]:
from torch import Tensor
from torch.nn import Transformer
import math

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class PositionalEncoding(nn.Module):
    def __init__(self,
                 emb_size: int,
                 dropout: float,
                 maxlen: int = 150):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: Tensor):
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])

class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

class Seq2SeqTransformer(nn.Module):
    def __init__(self,
                 num_encoder_layers: int,
                 num_decoder_layers: int,
                 emb_size: int,
                 nhead: int,
                 src_vocab_size: int,
                 tgt_vocab_size: int,
                 dim_feedforward: int = 512,
                 dropout: float = 0.1):
        super(Seq2SeqTransformer, self).__init__()
        self.transformer = Transformer(d_model=emb_size,
                                       nhead=nhead,
                                       num_encoder_layers=num_encoder_layers,
                                       num_decoder_layers=num_decoder_layers,
                                       dim_feedforward=dim_feedforward,
                                       dropout=dropout)
        self.generator = nn.Linear(emb_size, tgt_vocab_size)
        self.src_tok_emb = TokenEmbedding(src_vocab_size, emb_size)
        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
        self.positional_encoding = PositionalEncoding(
            emb_size, dropout=dropout)

    def forward(self,
                src: Tensor,
                trg: Tensor,
                src_mask: Tensor,
                tgt_mask: Tensor,
                src_padding_mask: Tensor,
                tgt_padding_mask: Tensor,
                memory_key_padding_mask: Tensor):
        src_emb = self.positional_encoding(self.src_tok_emb(src))
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
        outs = self.transformer(src_emb, tgt_emb, src_mask, tgt_mask, None,
                                src_padding_mask, tgt_padding_mask, memory_key_padding_mask)
        x = self.generator(outs)
        return x

    def encode(self, src: Tensor, src_mask: Tensor):
        return self.transformer.encoder(self.positional_encoding(
                            self.src_tok_emb(src)), src_mask)

    def decode(self, tgt: Tensor, memory: Tensor, tgt_mask: Tensor):
        return self.transformer.decoder(self.positional_encoding(
                          self.tgt_tok_emb(tgt)), memory,
                          tgt_mask)

def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask


def create_mask(src, tgt, PAD_IDX):
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len),device=DEVICE).type(torch.bool)

    src_padding_mask = (src == PAD_IDX).transpose(0, 1)
    tgt_padding_mask = (tgt == PAD_IDX).transpose(0, 1)

    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

**Функции для обучения модели и её оценки**

In [16]:
from time import time
import numpy as np
from tqdm.notebook import tqdm

def train(model, iterator, optimizer, criterion, print_every=500):

    global PAD_IDX
    epoch_loss = []
    ac = []
    model.train()

    for i, (texts_en, texts_ru) in tqdm(enumerate(iterator)):
        texts_en = texts_en.T.to(DEVICE)
        texts_ru = texts_ru.T.to(DEVICE)

        texts_ru_input = texts_ru[:-1, :]

        (texts_en_mask, texts_ru_mask,
        texts_en_padding_mask, texts_ru_padding_mask) = create_mask(texts_en, texts_ru_input, PAD_IDX)
        logits = model(texts_en, texts_ru_input, texts_en_mask, texts_ru_mask,
                       texts_en_padding_mask, texts_ru_padding_mask, texts_en_padding_mask)
        optimizer.zero_grad()

        texts_ru_out = texts_ru[1:, :]
        loss = criterion(logits.reshape(-1, logits.shape[-1]), texts_ru_out.reshape(-1))
        loss.backward()
        optimizer.step()
        epoch_loss.append(loss.item())

        if not (i+1) % print_every:
            print(f'Loss: {np.mean(epoch_loss)};')

    return np.mean(epoch_loss)


def evaluate(model, iterator, criterion):

    global PAD_IDX
    epoch_loss = []
    epoch_f1 = []
    model.eval()

    with torch.no_grad():
        for i, (texts_en, texts_ru) in enumerate(iterator):
            texts_en = texts_en.T.to(DEVICE)
            texts_ru = texts_ru.T.to(DEVICE)

            texts_ru_input = texts_ru[:-1, :]

            (texts_en_mask, texts_ru_mask,
            texts_en_padding_mask, texts_ru_padding_mask) = create_mask(texts_en, texts_ru_input, PAD_IDX)

            logits = model(texts_en, texts_ru_input, texts_en_mask, texts_ru_mask,
                           texts_en_padding_mask, texts_ru_padding_mask, texts_en_padding_mask)


            texts_ru_out = texts_ru[1:, :]
            loss = criterion(logits.reshape(-1, logits.shape[-1]), texts_ru_out.reshape(-1))
            epoch_loss.append(loss.item())

    return np.mean(epoch_loss)

**Подготовка данных**

In [17]:
def encode(sents, tokenizer, max_len):
    for text in sents:
        yield [tokenizer.token_to_id('[CLS]')] + tokenizer.encode(text).ids[:max_len] + [tokenizer.token_to_id('[SEP]')]

In [18]:
PAD_IDX = tokenizer_ru.token_to_id('[PAD]')
max_len_en, max_len_ru = 30, 35
PAD_IDX

3

In [19]:
X_en_train = list(encode(en_sents_train, tokenizer_en, max_len_en))
X_ru_train = list(encode(ru_sents_train, tokenizer_ru, max_len_ru))

X_en_valid = list(encode(en_sents_dev, tokenizer_en, max_len_en))
X_ru_valid = list(encode(ru_sents_dev, tokenizer_ru, max_len_ru))

In [20]:
training_set = Dataset(X_en_train, X_ru_train, PAD_IDX)
training_generator = torch.utils.data.DataLoader(training_set, batch_size=200, shuffle=True, )

valid_set = Dataset(X_en_valid, X_ru_valid, PAD_IDX)
valid_generator = torch.utils.data.DataLoader(valid_set, batch_size=200, shuffle=True)

**Параметры обучения модели**

In [21]:
torch.manual_seed(0)

EN_VOCAB_SIZE = tokenizer_en.get_vocab_size()
RU_VOCAB_SIZE = tokenizer_ru.get_vocab_size()

EMB_SIZE = 256
NHEAD = 8
FFN_HID_DIM = 512
NUM_ENCODER_LAYERS = 2
NUM_DECODER_LAYERS = 2

transformer = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE,
                                 NHEAD, EN_VOCAB_SIZE, RU_VOCAB_SIZE, FFN_HID_DIM)

for p in transformer.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

transformer = transformer.to(DEVICE)

loss_fn = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX).to(DEVICE)

optimizer = torch.optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

**Обучение модели**

In [22]:
from timeit import default_timer as timer

NUM_EPOCHS = 7
losses = {'train loss': [], 'val loss': []}

for epoch in range(1, NUM_EPOCHS+1):
    start_time = timer()
    train_loss = train(transformer, training_generator, optimizer, loss_fn)
    end_time = timer()
    val_loss = evaluate(transformer, valid_generator, loss_fn)

    if not losses['train loss']:
        min_val = val_loss
        print(f'First epoch - {val_loss}, saving model..')
        torch.save(transformer, 'model.pt')

    elif val_loss < min_val:
        min_val = min(losses['val loss'])
        print(f'Improved from {min_val} to {val_loss}, saving model..')
        torch.save(transformer, 'model.pt')

    losses['train loss'].append(train_loss)
    losses['val loss'].append(val_loss)

    print((f"Epoch: {epoch}, Train loss: {train_loss:.3f}, Val loss: {val_loss:.3f}, \
           "f"Epoch time={(end_time-start_time):.3f}s"))

0it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:4999: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


Loss: 7.779774409294128;
Loss: 7.18678768157959;
Loss: 6.8568265368143715;
Loss: 6.623233200073242;
Loss: 6.442462522506714;
Loss: 6.291134342511495;
Loss: 6.161977711677551;
Loss: 6.048552580475807;
Loss: 5.94817230881585;
Loss: 5.855449386405945;
First epoch - 5.030548763275147, saving model..
Epoch: 1, Train loss: 5.855, Val loss: 5.031,            Epoch time=901.042s


0it [00:00, ?it/s]

Loss: 4.879928792953491;
Loss: 4.839105271816254;
Loss: 4.794305472373963;
Loss: 4.751869009256363;
Loss: 4.711366484451294;
Loss: 4.6728480102221175;
Loss: 4.6349241143635345;
Loss: 4.59998843473196;
Loss: 4.563213287724389;
Loss: 4.528142247772217;
Improved from 5.030548763275147 to 4.1624836921691895, saving model..
Epoch: 2, Train loss: 4.528, Val loss: 4.162,            Epoch time=909.021s


...

Loss: 3.346145844459534;
Loss: 3.34584005689621;
Loss: 3.3429053155581157;
Loss: 3.3364502733945844;
Loss: 3.330620180797577;
Loss: 3.3248773081302643;
Loss: 3.318980755056654;
Loss: 3.314111970603466;
Loss: 3.3076032914585536;
Loss: 3.302200672483444;
Improved from 3.3677597045898438 to 3.1878969192504885, saving model..
Epoch: 5, Train loss: 3.302, Val loss: 3.188,            Epoch time=910.136s


0it [00:00, ?it/s]

Loss: 3.157115840435028;
Loss: 3.1556942102909087;
Loss: 3.156623980998993;
Loss: 3.154150164842606;
Loss: 3.1515759660720826;
Loss: 3.149160017649333;
Loss: 3.1471013903617857;
Loss: 3.144620652139187;
Loss: 3.1416941641701595;
Loss: 3.1381762807846068;
Improved from 3.1878969192504885 to 3.063573718070984, saving model..
Epoch: 6, Train loss: 3.138, Val loss: 3.064,            Epoch time=911.469s


0it [00:00, ?it/s]

Loss: 3.0088712730407714;
Loss: 3.0158162043094636;
Loss: 3.0237640374501544;
Loss: 3.0237956490516664;
Loss: 3.0237798171043395;
Loss: 3.0228738323052724;
Loss: 3.02228182383946;
Loss: 3.0214779188632965;
Loss: 3.021161272737715;
Loss: 3.0204352233409884;
Improved from 3.063573718070984 to 2.9827221393585206, saving model..
Epoch: 7, Train loss: 3.020, Val loss: 2.983,            Epoch time=911.436s


**Оценка модели**

In [44]:
def translate(text: str) -> str:
    global PAD_IDX, tokenizer_en, tokenizer_ru, transformer

    input_ids = [tokenizer_en.token_to_id('[CLS]')] + tokenizer_en.encode(text).ids[:max_len_en] + [tokenizer_en.token_to_id('[SEP]')]
    output_ids = [tokenizer_ru.token_to_id('[CLS]')]

    input_ids_pad = torch.nn.utils.rnn.pad_sequence([torch.LongTensor(input_ids)]).to(DEVICE)
    output_ids_pad = torch.nn.utils.rnn.pad_sequence([torch.LongTensor(output_ids)]).to(DEVICE)

    (texts_en_mask, texts_ru_mask,
    texts_en_padding_mask, texts_ru_padding_mask) = create_mask(input_ids_pad, output_ids_pad, PAD_IDX)
    logits = transformer(input_ids_pad, output_ids_pad, texts_en_mask, texts_ru_mask,
                    texts_en_padding_mask, texts_ru_padding_mask, texts_en_padding_mask)
    pred = logits.argmax(2).item()

    while pred not in [tokenizer_ru.token_to_id('[SEP]'), tokenizer_ru.token_to_id('[PAD]')]:
        output_ids.append(pred)
        output_ids_pad = torch.nn.utils.rnn.pad_sequence([torch.LongTensor(output_ids)]).to(DEVICE)

        (texts_en_mask, texts_ru_mask,
        texts_en_padding_mask, texts_ru_padding_mask) = create_mask(input_ids_pad, output_ids_pad, PAD_IDX)
        logits = transformer(input_ids_pad, output_ids_pad, texts_en_mask, texts_ru_mask,
                        texts_en_padding_mask, texts_ru_padding_mask, texts_en_padding_mask)
        pred = logits.argmax(2)[-1].item()

    return (' '.join([tokenizer_ru.id_to_token(i) for i in output_ids[1:]]))

In [28]:
translate('Please translate this sentence into Russian')

/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:4999: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


'Пожалуйста , перевести это предложение в россий ском'

**Оценка модели**

In [35]:
en_sents_test[:5], ru_sents_test[:5]

(['If you only stay there.',
  "I don't know how you do it, Pop, carrying these boxes around every day.",
  'We might have a slight edge in mediation.',
  'How long is it going to take you to get him what he needs?',
  "On 1 April President of the Nagorno Karabagh Republic Bako Sahakyan met head of the General Staff of the Republic of Armenia's Armed forces colonel-general Yuri Khachaturov."],
 ['Только бы не вылететь.',
  'И как ты только справляешься, папа, таская эти коробки взад-вперед целый день.',
  'Возможно, у нас есть небольшое преимущество в переговорах.',
  'Сколько времени вы будете делать то, что ему нужно?',
  '1 апреля Президент НКР Бако Саакян принял начальника Генштаба Вооруженных сил Республики Армения генерал-полковника Юрия Хачатурова.'])

In [36]:
len(en_sents_test)

2000

In [38]:
ru_sent_translation = [translate(en_s) for en_s in tqdm(en_sents_test)]

  0%|          | 0/2000 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:4999: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


In [ ]:
import nltk
bleus = []

for i, t in tqdm(enumerate(ru_sent_translation)):
    reference = tokenizer_ru.encode(t).tokens
    hypothesis = tokenizer_ru.encode(ru_sents_test[i]).tokens

    bleus.append(nltk.translate.bleu_score.sentence_bleu([reference], hypothesis,  auto_reweigh=True))

**Средний скор bleu**

In [42]:
(sum(bleus)/len(bleus))*100

11.89192063814854

In [48]:
for en, s, tr in zip(en_sents_test[:10], ru_sents_test[:10], ru_sent_translation[:10]):
    print(en, '===', s, '===', tr)

If you only stay there. === Только бы не вылететь. === Если вы оста нетесь там .
I don't know how you do it, Pop, carrying these boxes around every day. === И как ты только справляешься, папа, таская эти коробки взад-вперед целый день. === Я не знаю , как ты это делаешь , По ру ж , за нести эти короб ки каждый день .
We might have a slight edge in mediation. === Возможно, у нас есть небольшое преимущество в переговорах. === Мы могли бы иметь небольшой край ний край ний характер .
How long is it going to take you to get him what he needs? === Сколько времени вы будете делать то, что ему нужно? === Сколько тебе нужно , чтобы он мог получить ему , что ему нужно ?
On 1 April President of the Nagorno Karabagh Republic Bako Sahakyan met head of the General Staff of the Republic of Armenia's Armed forces colonel-general Yuri Khachaturov. === 1 апреля Президент НКР Бако Саакян принял начальника Генштаба Вооруженных сил Республики Армения генерал-полковника Юрия Хачатурова. === 1 апреля Президе

По первым примерам видно, что модель скорее переводит буквально. Возможно, нужно большее время обучения для более высокоуровневого перевода.

Кроме того, данные несколько зашумлены, что видно в примерах с датами. Поэтому их нужно было бы почистить.

Другие ошибки вроде неправильно сопоставленных реплик (см. пример с *куриным супом*) исправить сложнее.


## Задание 2 (2 балла).
Прочитайте главу про машинный перевод у Журафски и Маннига - https://web.stanford.edu/~jurafsky/slp3/10.pdf
Ответьте своими словами в чем заключается техника back translation? Для чего она применяется и что позволяет получить? Опишите по шагам как его применить к паре en-ru на данных из семинара.

Back translation используется для аугментации данных и заключается в том, что сначала тренируется одна модель, которая учится предсказывать перевод с исходного на целевой язык. Затем предсказывается ряд переводов тренировочных данных на целевой язык и обратно. Полученные данные становятся частью всех данных, на которых и обучается финальная модель.

1. Обучаем наш seq2seq трансформер en-ru.
2. Предсказываем перевод для предложений обучающего корпуса.
3. Переводим их обратно на английский:
    * либо обученным на том же корпусе seq2seq трансформер ru-en,
    * либо другими инструментами автоматического перевода, например, api google translate
4. Добавляем полученные примеры в обучающий корпус.
5. Дообучаем финальный трансформер на всех данных.

Такии образом, мы должны некоторым образом преодолеть слишком дословный перевод трансформерной модели.